In [ ]:
from collections import Counter
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
from copy import deepcopy
import pandas as pd
from sklearn.preprocessing import StandardScaler
import copy
from itertools import combinations

In [ ]:
df = 'your_train_set.csv' #your train_set.csv

df = pd.read_csv(df)

numeric_columns = []
integer_columns = []

for col in df.columns:
    if df[col].dtype == 'float64':
        numeric_columns.append(col)
    elif df[col].dtype == 'int64':
        integer_columns.append(col)
        
import pickle

nome_do_arquivo_pickle = 'triselect.pickle'

with open(nome_do_arquivo_pickle, 'rb') as f:
    triselect = pickle.load(f)

    
n_classes=2 #number of classes

In [ ]:
with open('transactions_all.txt', 'r') as file:
    content = file.read()

import pandas as pd


lines = content.strip().split('\n')

antecedents = []
consequents = []

for line in lines:
    elements = line.split()
    antecedents.append(' '.join(elements[:-1]))
    consequents.append(elements[-1])

rul = pd.DataFrame({
    'antecedents': antecedents,
    'consequents': consequents
})

import pandas as pd

def extract_rules_data(dataframe):
    rules_dict = {}

    for index, row in dataframe.iterrows():
        antecedents = row['antecedents'].split()
        consequent = row['consequents']

        if consequent not in rules_dict:
            rules_dict[consequent] = []
        rules_dict[consequent].append(antecedents)

    return rules_dict

rules_data = extract_rules_data(rul)

In [ ]:
def classify_rules(rules_data, numeric_columns, integer_columns):
    result_dict = {}

    for consequent, antecedents_list in rules_data.items():
        classification_list = []

        for antecedents in antecedents_list:
            if all(element in numeric_columns for element in antecedents):
                classification_list.append({'antecedents': antecedents, 'classification': 'N'})
            elif all(element in integer_columns for element in antecedents):
                classification_list.append({'antecedents': antecedents, 'classification': 'C'})
            else:
                classification_list.append({'antecedents': antecedents, 'classification': 'M'})

        result_dict[list(consequent)[0]] = classification_list

    return result_dict

resultrules = classify_rules(rules_data, numeric_columns, integer_columns)

In [ ]:
def select_triclusters(triselect, result):
    selected_triclusters = {}

    for key, antecedents_list in result.items():
        key = int(key)  
        selected_triclusters[key] = {}

        for entry in antecedents_list:
            antecedents = entry['antecedents']
            classification = entry['classification']

            selected_triclusters[key][tuple(antecedents)] = []

            for tricluster_key, tricluster_df in triselect.items():
                tricluster_columns = set(tricluster_df.columns)
                moda = tricluster_df[' class'].mode()[0]
                total_elementos = len(tricluster_df[' class'])
                num_moda = (tricluster_df[' class'] == moda).sum()

                if set(antecedents).issubset(tricluster_columns):
                    selected_triclusters[key][tuple(antecedents)].append({
                        'tricluster_key': tricluster_key,
                        'tricluster_df': tricluster_df,
                        'classification': classification,
                        'ratio':num_moda/total_elementos
                    })

    return selected_triclusters


selected_triclusters1 = select_triclusters(triselect, resultrules)

In [ ]:
import pandas as pd

def filter_triclusters(selected_triclusters):
    filtered_triclusters = {}

    for key, antecedents_dict in selected_triclusters.items():
        filtered_triclusters[key] = {}

        for antecedents, triclusters_list in antecedents_dict.items():
            filtered_triclusters[key][antecedents] = []

            for tricluster in triclusters_list:
                tricluster_df = tricluster['tricluster_df']
                majority_class = tricluster_df[' class'].mode().values[0]

               
                key_as_int = int(key)

               
                if majority_class == key_as_int:
                    filtered_triclusters[key][antecedents].append(tricluster)

    return filtered_triclusters

selected_triclusters = filter_triclusters(selected_triclusters1)

In [ ]:
def filter_by_class(selected_triclusters):
    filtered_triclusters = {}

    for key, antecedents_dict in selected_triclusters.items():
        filtered_triclusters[key] = {}

        for antecedents, triclusters_list in antecedents_dict.items():
            filtered_triclusters[key][antecedents] = []

            for tricluster in triclusters_list:
                tricluster_df = tricluster['tricluster_df']
                target_class = tricluster['classification']

               
                key_as_int = int(key)

               
                filtered_df = tricluster_df[tricluster_df[' class'] == key_as_int]

              
                tricluster_copy = tricluster.copy()
                tricluster_copy['tricluster_df'] = filtered_df
                filtered_triclusters[key][antecedents].append(tricluster_copy)

    return filtered_triclusters

filtered_triclusters = filter_by_class(selected_triclusters)

In [ ]:
def remove_extra_columns(selected_triclusters):
    modified_triclusters = {}

    for key, antecedents_dict in selected_triclusters.items():
        modified_triclusters[key] = {}

        for antecedents, triclusters_list in antecedents_dict.items():
            modified_triclusters[key][antecedents] = []

            for tricluster in triclusters_list:
                tricluster_df = tricluster['tricluster_df']
                columns_to_keep = list(tricluster_df.columns)

                
                columns_to_remove = [col for col in columns_to_keep if col not in antecedents and col != 'X']
                tricluster_df_modified = tricluster_df.drop(columns=columns_to_remove)

               
                tricluster_copy = tricluster.copy()
                tricluster_copy['tricluster_df'] = tricluster_df_modified
                modified_triclusters[key][antecedents].append(tricluster_copy)

    return modified_triclusters

modified_triclusters = remove_extra_columns(filtered_triclusters)

In [ ]:
def organize_triclusters(modified_triclusters):
    organized_triclusters = {}

    for outer_key, inner_dict in modified_triclusters.items():
        organized_triclusters[outer_key] = {}

        for tuplo, triclusters_list in inner_dict.items():
            for tricluster in triclusters_list:
                classification = tricluster['classification']
                tricluster_key = tricluster['tricluster_key']
                tricluster_df = tricluster['tricluster_df']
                ratio=tricluster['ratio']


                new_key = (tuplo, classification)

                if new_key not in organized_triclusters[outer_key]:
                    organized_triclusters[outer_key][new_key] = []

                organized_triclusters[outer_key][new_key].append({'tricluster_key': tricluster_key, 'tricluster_df': tricluster_df,'ratio':ratio})

    return organized_triclusters

triclusters_ready=organize_triclusters(modified_triclusters)

In [ ]:
def RulesN(dictionary):
    filtered_dict = {}
    
    for key, value in dictionary.items():
        filtered_value = {sub_key: sub_value for sub_key, sub_value in value.items() if sub_key[1] == 'N'}
        
        if filtered_value:
            filtered_dict[key] = filtered_value
    
    return filtered_dict


rulesN = RulesN(triclusters_ready)
rulesN

In [ ]:
def RulesM(dictionary):
    filtered_dict = {}
    
    for key, value in dictionary.items():
        filtered_value = {sub_key: sub_value for sub_key, sub_value in value.items() if sub_key[1] == 'M'}
        
        if filtered_value:
            filtered_dict[key] = filtered_value
    
    return filtered_dict


rulesM = RulesM(triclusters_ready)
rulesM

In [ ]:
def RulesC(dictionary):
    filtered_dict = {}
    
    for key, value in dictionary.items():
        filtered_value = {sub_key: sub_value for sub_key, sub_value in value.items() if sub_key[1] == 'C'}
        
        if filtered_value:
            filtered_dict[key] = filtered_value
    
    return filtered_dict


rulesC = RulesC(triclusters_ready)
rulesC

In [ ]:
def split_dataframes(rulesM, numeric_columns, integer_columns):
    rulesMC = {}
    rulesMN = {}

    for key, value in rulesM.items():
        rulesMC[key] = {}
        rulesMN[key] = {}

        for sub_key, tricluster_list in value.items():
            rulesMC[key][sub_key] = []
            rulesMN[key][sub_key] = []

            for tricluster in tricluster_list:
                tricluster_key = tricluster['tricluster_key']
                tricluster_df = tricluster['tricluster_df']
                ratio=tricluster['ratio']

                
                numeric_df = tricluster_df[tricluster_df.columns.intersection(integer_columns + ['X'])]
                rulesMC[key][sub_key].append({'tricluster_key': tricluster_key, 'tricluster_df': numeric_df,'ratio':ratio})

                
                integer_df = tricluster_df[tricluster_df.columns.intersection(numeric_columns + ['X'])]
                rulesMN[key][sub_key].append({'tricluster_key': tricluster_key, 'tricluster_df': integer_df,'ratio':ratio})
    
    new_rulesMN = {}

    for key, value in rulesMN.items():
        new_key = key
        new_value = {}
    
        for inner_key, inner_value_list in value.items():
            new_inner_key = (inner_key[0], 'MN')
            new_inner_value_list = []

            for inner_value in inner_value_list:
                new_inner_value = {
                    'tricluster_key': inner_value['tricluster_key'],
                    'tricluster_df': inner_value['tricluster_df'],
                    'ratio':inner_value['ratio']
                }
                new_inner_value_list.append(new_inner_value)

            new_value[new_inner_key] = new_inner_value_list

        new_rulesMN[new_key] = new_value
        
    new_rulesMC={}
    for key, value in rulesMC.items():
        new_key = key
        new_value = {}
    
        for inner_key, inner_value_list in value.items():
            new_inner_key = (inner_key[0], 'MC')
            new_inner_value_list = []

            for inner_value in inner_value_list:
                new_inner_value = {
                    'tricluster_key': inner_value['tricluster_key'],
                    'tricluster_df': inner_value['tricluster_df'],
                    'ratio':inner_value['ratio']
                }
                new_inner_value_list.append(new_inner_value)

            new_value[new_inner_key] = new_inner_value_list

        new_rulesMC[new_key] = new_value
        
    return new_rulesMC, new_rulesMN

rulesMC, rulesMN = split_dataframes(rulesM, numeric_columns, integer_columns)

rulesMC

In [ ]:
def calculate_jaccard_similarity(row1, row2):
    list1 = list(row1.values[1:])  
    list2 = list(row2.values[1:])  
    
    intersection = 0
    union = len(list1)

    for value1, value2 in zip(list1, list2):
        if value1 == value2:
            intersection += 1

    jaccard_similarity = intersection / union if union != 0 else 0
    return jaccard_similarity

def calculate_average_jaccard_similarity(tricluster_df):
    objects = tricluster_df['X'].unique()
    pairs = list(combinations(objects, 2))

    similarities = []

    for pair in pairs:
        obj1, obj2 = pair
        obj1_row = tricluster_df[tricluster_df['X'] == obj1].iloc[0]
        obj2_row = tricluster_df[tricluster_df['X'] == obj2].iloc[0]

        jaccard_similarity = calculate_jaccard_similarity(obj1_row, obj2_row)
        similarities.append(jaccard_similarity)

    if similarities:
        average_similarity = sum(similarities) / len(similarities)
    else:
        average_similarity = 0

    return average_similarity

def calculate_and_assign_jaccard_similarity(rulesMCfit):
    new_rulesMCfit = {}

    for key, value in rulesMCfit.items():
        new_key = key
        new_value = {}

        for inner_key, inner_value_list in value.items():
            new_inner_key = (inner_key[0], 'MC')
            new_inner_value_list = []

            for inner_value in inner_value_list:
                tricluster_key = inner_value['tricluster_key']
                tricluster_df = inner_value['tricluster_df']
                ratio=inner_value['ratio']

                average_similarity = calculate_average_jaccard_similarity(tricluster_df)

                new_inner_value = {
                    'tricluster_key': tricluster_key,
                    'tricluster_df': tricluster_df,
                    'tol_min_por_cento': average_similarity,
                    'ratio':ratio
                }
                new_inner_value_list.append(new_inner_value)

            new_value[new_inner_key] = new_inner_value_list

        new_rulesMCfit[new_key] = new_value

    return new_rulesMCfit


rulesMCfit2 = calculate_and_assign_jaccard_similarity(rulesMC)

def remove_dictionaries_with_zero_tol(ruleMCfit2, min_threshold):
    rulesMCfit = {}
    for key, value in ruleMCfit2.items():
        for sub_key, sub_value in value.items():
            filtered_values = []
            for item in sub_value:
                if item['tol_min_por_cento'] > min_threshold:
                    filtered_values.append(item)
            if filtered_values:
                if key not in rulesMCfit:
                    rulesMCfit[key] = {}
                rulesMCfit[key][sub_key] = filtered_values
    return rulesMCfit


rulesMCfit = remove_dictionaries_with_zero_tol(rulesMCfit2, 0)
rulesMCfit

In [ ]:
def calculate_and_assign_jaccard_similarity(rulesMCfitu):
    rulesMCfit = copy.deepcopy(rulesMCfitu)
    new_rulesMCfit = {}

    for key, value in rulesMCfit.items():
        new_key = key
        new_value = {}

        for inner_key, inner_value_list in value.items():
            new_inner_key = (inner_key[0], 'C')
            new_inner_value_list = []

            for inner_value in inner_value_list:
                tricluster_key = inner_value['tricluster_key']
                tricluster_df = inner_value['tricluster_df']
                ratio=inner_value['ratio']

                average_similarity = calculate_average_jaccard_similarity(tricluster_df)

                new_inner_value = {
                    'tricluster_key': tricluster_key,
                    'tricluster_df': tricluster_df,
                    'tol_min_por_cento': average_similarity,
                    'ratio':ratio
                }
                new_inner_value_list.append(new_inner_value)

            new_value[new_inner_key] = new_inner_value_list

        new_rulesMCfit[new_key] = new_value

    return new_rulesMCfit



def remove_dictionaries_with_zero_tol(ruleMCfit2, min_threshold):
    rulesMCfit = {}
    for key, value in ruleMCfit2.items():
        for sub_key, sub_value in value.items():
            filtered_values = []
            for item in sub_value:
                if item['tol_min_por_cento'] > min_threshold:
                    filtered_values.append(item)
            if filtered_values:
                if key not in rulesMCfit:
                    rulesMCfit[key] = {}
                rulesMCfit[key][sub_key] = filtered_values
    return rulesMCfit




rulesCfit2 = calculate_and_assign_jaccard_similarity(rulesC)
rulesCfit = remove_dictionaries_with_zero_tol(rulesCfit2, 0)
rulesCfit

In [ ]:
import pandas as pd

def calculate_mean(df):
    
    df = df.drop(columns=['X'])
    
    mean_values = df.mean(axis=0)
    return mean_values

def calculate_means_in_rulesNtype(rulesN):
    result = {}
    for key, value in rulesN.items():
        result[key] = {}
        for condition, sub_dict in value.items():
            result[key][condition] = []
            for sub_item in sub_dict:
                tricluster_key = sub_item['tricluster_key']
                tricluster_df = sub_item['tricluster_df']
                ratio=sub_item['ratio']
                mean_values = calculate_mean(tricluster_df)
                result[key][condition].append({'tricluster_key': tricluster_key, 'mean_values': mean_values,'ratio':ratio})
    return result


rulesNmean = calculate_means_in_rulesNtype(rulesN)
rulesNmean


In [ ]:
rulesMNmean = calculate_means_in_rulesNtype(rulesMN)
rulesMNmean

In [ ]:
def adicionar_coordenada_z(dicionario):
    novo_dicionario = {}

    for chave, valor in dicionario.items():
        novo_valor = {}
        for subchave, lista_valores in valor.items():
            novo_item = []
            for item in lista_valores:
                novo_subitem = {}
                novo_subitem['tricluster_key'] = item['tricluster_key']
                novo_subitem['mean_values'] = {}
                novo_subitem['ratio']=item['ratio']
                for coluna, valor in item['mean_values'].items():
                    partes_coluna = coluna.split('z')
                    coordenada_z = int(partes_coluna[1])
                    novo_subitem['mean_values'][coluna] = (valor, coordenada_z)
                novo_item.append(novo_subitem)
            novo_valor[subchave] = novo_item
        novo_dicionario[chave] = novo_valor

    return novo_dicionario



rulesNmeanZ=adicionar_coordenada_z(rulesNmean)
rulesMNmeanZ=adicionar_coordenada_z(rulesMNmean)

In [ ]:
rulesNmeanZ
#rulesMNmeanZ

In [ ]:
def agrupar_por_y_e_z(dicionario):
    novo_dicionario = {}

    for chave, valor in dicionario.items():
        novo_valor = {}
        for subchave, lista_valores in valor.items():
            novo_item = []
            for item in lista_valores:
                novo_subitem = {}
                novo_subitem['tricluster_key'] = item['tricluster_key']
                novo_subitem['ratio']=item['ratio']

                grupos = {}
                for coluna, (valor, z) in item['mean_values'].items():
                    y = coluna.split('z')[0]
                    if y not in grupos:
                        grupos[y] = []
                    grupos[y].append((coluna, (valor, z)))

                for y, colunas in grupos.items():
                    grupo = {}
                    grupo.update({'mean_values': dict(colunas)})
                    grupo['tolerancia'] = []  
                    grupo['coef_ind'] = []  
                    novo_subitem[y] = grupo

                novo_item.append(novo_subitem)
            novo_valor[subchave] = novo_item
        novo_dicionario[chave] = novo_valor

    return novo_dicionario


rulesNmeanZgroup=agrupar_por_y_e_z(rulesNmeanZ)
rulesMNmeanZgroup=agrupar_por_y_e_z(rulesMNmeanZ)

In [ ]:
rulesNmeanZgroup
#rulesMNmeanZgroup

In [ ]:
def separar_por_tricluster(rulesNmeanZgroup):
    tricluster_dict_list = []
    
    for key, value in rulesNmeanZgroup.items():
        for item_key, item_value in value.items():
            for item in item_value:
                tricluster_dict_list.append({key: {item_key: [item]}})
    
    return tricluster_dict_list


rulesNlist = separar_por_tricluster(rulesNmeanZgroup)
rulesMNlist = separar_por_tricluster(rulesMNmeanZgroup)


In [ ]:
rulesNlist
#rulesMNlist

In [ ]:
def create_variablesN(result):
    variables = []
    current_key = None
    current_subkey = None
    current_subkey_value = None
    subkey_counter = {}
    previous_key = None
    previous_subkey_value = None
    Y = 1

    for d in result:
        for key, value in d.items():
            if current_key != key:
                Y = 0

            current_key = key
            previous_key = key
            for subkey, subvalue in value.items():
                if subkey != current_subkey:
                    Y += 1
                    current_subkey = subkey
                    current_subkey_value = subkey[0]  
                    previous_subkey_value = current_subkey_value
                for item in subvalue:
                    tricluster_key = item['tricluster_key']
                    variable_name = f'rulesN{current_key}{Y}{tricluster_key}'
                    variables.append(variable_name)
    return variables

rulesNlistvar = create_variablesN(rulesNlist)

def create_variablesMN(result):
    variables = []
    current_key = None
    current_subkey = None
    current_subkey_value = None
    subkey_counter = {}
    previous_key = None
    previous_subkey_value = None
    Y = 1

    for d in result:
        for key, value in d.items():
            if current_key != key:
                Y = 0

            current_key = key
            previous_key = key
            for subkey, subvalue in value.items():
                if subkey != current_subkey:
                    Y += 1
                    current_subkey = subkey
                    current_subkey_value = subkey[0] 
                    previous_subkey_value = current_subkey_value
                for item in subvalue:
                    tricluster_key = item['tricluster_key']
                    variable_name = f'rulesMN{current_key}{Y}{tricluster_key}'
                    variables.append(variable_name)
    return variables

rulesMNlistvar = create_variablesMN(rulesMNlist)

In [ ]:
rulesNlistvar
#rulesMNlistvar

In [ ]:
def create_variablesN(rulesNlist, rulesNlistvar):
    
    rulesN = []

    
    for i, var_name in enumerate(rulesNlistvar):
        
        globals()[var_name] = rulesNlist[i]
        
        
        rulesN.append(var_name)
    
    return rulesN

rulesNlistvar = create_variablesN(rulesNlist, rulesNlistvar)

def create_variablesMN(rulesMNlist, rulesMNlistvar):
    
    rulesMN = []

    
    for i, var_name in enumerate(rulesMNlistvar):
        
        globals()[var_name] = rulesMNlist[i]
        
        
        rulesMN.append(var_name)
    
    return rulesMN

rulesMNlistvar = create_variablesMN(rulesMNlist, rulesMNlistvar)

In [ ]:
import re

def agrupar_variaveis(lista_variaveis):
    
    agrupadas = {}
    
    
    padrao = r'(\d+)tctri'
    
    for var in lista_variaveis:
        
        match = re.search(padrao, var)
        if match:
            sequencia = match.group(1)
            
            agrupadas.setdefault(sequencia, []).append(var)
    
    
    lista_agrupada = list(agrupadas.values())
    
    return lista_agrupada

rulesNlistlist = agrupar_variaveis(rulesNlistvar)
rulesMNlistlist = agrupar_variaveis(rulesMNlistvar)

In [ ]:
rulesNlistvar
#rulesMNlistlist

In [ ]:
def criar_listas(rulesNlistlist):
    lista_de_variaveis = []
    for lista in rulesNlistlist:
        
        indice_tctri = lista[0].find("tctri")
        if indice_tctri != -1:  
            
            nome_variavel = lista[0][:indice_tctri]
            
            globals()[nome_variavel] = lista
            lista_de_variaveis.append(nome_variavel)
    return lista_de_variaveis


rulesNlistfit = criar_listas(rulesNlistlist)
rulesMNlistfit = criar_listas(rulesMNlistlist)


In [ ]:
rulesNlistfit
#rulesMNlistfit

In [ ]:
def separar_por_tricluster(rulesMCfitu):
    rulesMCfit=copy.deepcopy(rulesMCfitu)
    tricluster_dict_list = []

    for key, inner_dict in rulesMCfit.items():
        for nested_key, nested_value in inner_dict.items():
            for item in nested_value:
                tricluster_key = item['tricluster_key']
                tricluster_dict = {
                    key: {
                        tricluster_key: {
                            nested_key: [item]
                        }
                    }
                }
                tricluster_dict_list.append(tricluster_dict)

    return tricluster_dict_list

rulesMClist = separar_por_tricluster(rulesMCfit)
rulesClist = separar_por_tricluster(rulesCfit)

In [ ]:
rulesClist

In [ ]:
def create_variable_names(rulesMClist):
    variable_names = []
    prev_outer_key = None
    prev_tricluster_key = None
    counter = 1

    for d in rulesMClist:
        outer_key = list(d.keys())[0]
        inner_key = list(d[outer_key].keys())[0]
        tricluster_key = list(d[outer_key][inner_key].keys())[0]
        
        if prev_outer_key is None or outer_key != prev_outer_key:
            counter = 1
        elif tricluster_key != prev_tricluster_key:
            counter += 1

        var_name = f"rulesMC{outer_key}{counter}{d[outer_key][inner_key][tricluster_key][0]['tricluster_key']}"
        variable_names.append(var_name)
        
        prev_outer_key = outer_key
        prev_tricluster_key = tricluster_key

    return variable_names

rulesMClistvar = create_variable_names(rulesMClist)

In [ ]:
def create_variable_namesC(rulesMClist):
    variable_names = []
    prev_outer_key = None
    prev_tricluster_key = None
    counter = 1

    for d in rulesMClist:
        outer_key = list(d.keys())[0]
        inner_key = list(d[outer_key].keys())[0]
        tricluster_key = list(d[outer_key][inner_key].keys())[0]
        
        if prev_outer_key is None or outer_key != prev_outer_key:
            counter = 1
        elif tricluster_key != prev_tricluster_key:
            counter += 1

        var_name = f"rulesC{outer_key}{counter}{d[outer_key][inner_key][tricluster_key][0]['tricluster_key']}"
        variable_names.append(var_name)
        
        prev_outer_key = outer_key
        prev_tricluster_key = tricluster_key

    return variable_names

rulesClistvar = create_variable_namesC(rulesClist)

In [ ]:
rulesClistvar

In [ ]:
def create_variables(rulesMClist, rulesMClistvar):
    
    rulesMC = []

    
    for i, var_name in enumerate(rulesMClistvar):
        
        globals()[var_name] = rulesMClist[i]
        
       
        rulesMC.append(var_name)
    
    return rulesMC

rulesMClistvar = create_variables(rulesMClist, rulesMClistvar)


In [ ]:
def create_variablesC(rulesMClist, rulesMClistvar):
    
    rulesMC = []

    
    for i, var_name in enumerate(rulesMClistvar):
        
        globals()[var_name] = rulesMClist[i]
        
        
        rulesMC.append(var_name)
    
    return rulesMC

rulesClistvar = create_variablesC(rulesClist, rulesClistvar)

In [ ]:
rulesMClistlist = agrupar_variaveis(rulesMClistvar)
rulesClistlist = agrupar_variaveis(rulesClistvar)

In [ ]:
rulesMClistfit = criar_listas(rulesMClistlist)
rulesClistfit = criar_listas(rulesClistlist)

In [ ]:
rulesMClistfit

In [ ]:
def chamarvar(listlistvar):
    for rule in listlistvar:
        variavel = globals()[rule]  
        
        return variavel
        
def chamarvarvar(variavel):
    dic=[]
    for rule in variavel:
        variavel = globals()[rule]  
        
        dic.append(variavel)
    return dic

In [ ]:
def calcular_r_quadrado(x, y):
    coeficientes = np.polyfit(x, y, 1)  
    y_pred = np.polyval(coeficientes, x)
    ss_total = np.sum((y - np.mean(y)) ** 2)
    ss_res = np.sum((y - y_pred) ** 2)
    r_quadrado = 1 - (ss_res / ss_total)
    return r_quadrado

def fit_polinomial(dicionario, grau_polinomio, toleranciau):
    novo_dicionario = {}
    
    for chave, valor in dicionario.items():
        novo_valor = {}
        for subchave, lista_valores in valor.items():
            novo_item = []
            for item in lista_valores:
                novo_subitem = {}
                novo_subitem['tricluster_key'] = item['tricluster_key']
                novo_subitem['ratio'] = item['ratio']
                
                for y, grupo in item.items():
                    if y != 'tricluster_key' and y != 'ratio':
                        coeficientes = {}
                        mean_values = grupo['mean_values']
                        x = [coord[1] for coord in mean_values.values()]
                        y_values = [coord[0] for coord in mean_values.values()]
                        
                        coef = np.polyfit(x, y_values, grau_polinomio)
                       
                        if grau_polinomio == 1:
                            coeficientes['a1'] = coef[0]
                       
                        else:
                            for i in range(grau_polinomio):
                                coeficientes[f'a{i+1}'] = coef[i]
                        
                        r_quadrado = calcular_r_quadrado(x, y_values)
                        tolerancia = 1 - r_quadrado + toleranciau
                        coeficientes['tolerancia'] = tolerancia  
                        novo_subitem[y] = {'coeficientes': coeficientes}
                
                novo_item.append(novo_subitem)
            novo_valor[subchave] = novo_item
        novo_dicionario[chave] = novo_valor
    
    return novo_dicionario


In [ ]:
def chamarvarfitpol(listlistvar):
    for rule in listlistvar:
        variavel = globals()[rule] 
        
        chamarvarvarfitpol(variavel)
        
def chamarvarvarfitpol(variavel):
    for rule in variavel:
        nome_variavel = rule  
        variavel_atual = globals()[rule]  
        resultado = fit_polinomial(variavel_atual, 1, 0.2) #0.2 is the Tolerance LR parameter
        globals()[nome_variavel] = resultado 

chamarvarfitpol(rulesNlistfit)
chamarvarfitpol(rulesMNlistfit)

In [ ]:
test = 'your_test_set.csv' #your test set .csv

test = pd.read_csv(test)
test

In [ ]:
scaler = StandardScaler()
test[numeric_columns] = scaler.fit_transform(test[numeric_columns])

test

In [ ]:
import pandas as pd


def extrair_lista_X(dataframe, resultrules,n_class):
    listX = dataframe['X'].tolist()
    listX1 = [[x, [[] for _ in range(n_class)]] for x in listX]
    for i, lis in enumerate (listX1):
        for j, lislis in enumerate (lis[1]):
            for (key, value) in resultrules.items():
                if int(key)==j:
                    for d in value:
                        if d['classification']=='N':
                            listX1[i][1][int(key)].append([])
                    for d in value:
                        if d['classification']=='C':
                            listX1[i][1][int(key)].append([])
                    for d in value:
                        if d['classification']=='M':
                            listX1[i][1][int(key)].append([[],[]])
                
    return listX1



class_results = extrair_lista_X(test, resultrules,n_classes)
class_results

## N e MN

In [ ]:
def extrair_colunas_e_valores(dataset, rulesNmeanZgroup_fit):
    testN = {}

    for index, row in dataset.iterrows():
        key = row['X']
        columns_values = {}
        group_counter = {}

        for rule_key, rule_values in rulesNmeanZgroup_fit.items():
            rule_columns = []
            for rule_index, rule in enumerate(rule_values):
                if set(rule[0]) <= set(dataset.columns[1:-1]):
                    rule_columns.extend(rule[0])
                    group_counter[rule_key] = group_counter.get(rule_key, 0) + 1
                    group_key = f"{rule_key}{group_counter[rule_key]}"
                    values = {}
                    for col in rule_columns:
                        values[col] = row[col]
                    columns_values[group_key] = values

        testN[key] = columns_values

    return testN

testN = extrair_colunas_e_valores(test, rulesNmeanZgroup)
testMN = extrair_colunas_e_valores(test, rulesMNmeanZgroup)

In [ ]:
testN

def remover_colunas_inteiras(testMN, integer_columns):
    new_testMN = {}

    for key, value in testMN.items():
        new_value = {}
        for sub_key, sub_value in value.items():
            new_sub_value = {col: val for col, val in sub_value.items() if col not in integer_columns}
            new_value[sub_key] = new_sub_value
        new_testMN[key] = new_value

    return new_testMN

testMN = remover_colunas_inteiras(testMN, integer_columns)
testMN

In [ ]:
def adicionar_coordenada_z_testN(testN):
    novo_testN = {}

    for chave, grupos in testN.items():
        novo_grupos = {}
        for grupo_chave, colunas_valores in grupos.items():
            novo_pontos = {}
            for coluna, valor in colunas_valores.items():
                partes_coluna = coluna.split('z')
                coordenada_z = int(partes_coluna[1])
                novo_pontos[coluna] = (valor, coordenada_z)

            novo_grupos[grupo_chave] = novo_pontos

        novo_testN[chave] = novo_grupos

    return novo_testN

testNz = adicionar_coordenada_z_testN(testN)
testMNz = adicionar_coordenada_z_testN(testMN)

In [ ]:
testNz
#testMNz

### N

In [ ]:
import copy

def z_segundo_y(test):
    testNz = copy.deepcopy(test)
    new_dict = {}
    
    for key, value in testNz.items():
        new_dict[key] = {}
        for key2 in value.keys():
            new_dict[key][key2] = {}
            
    for key, value in testNz.items():
        for key2, value2 in value.items():
            y_i=[]
            for key3 in value2.keys():
                padrao = r'y(.*?)z'
                encontrados = re.findall(padrao, key3)
                y_i.append(encontrados)
            y_i_ok = sorted(list(set([valor[0] for valor in y_i])),key=int)
            
            for i in y_i_ok:
                new_dict[key][key2][f"y{i}"] = {'fit': None, 'values': []}
                
                for key4, val4 in value2.items():
                    encontrados2 = re.findall(padrao, key4)
                    if encontrados2 and encontrados2[0] == i:
                        new_dict[key][key2][f"y{i}"]['values'].append(val4)

    return new_dict


testN = z_segundo_y(testNz)
testMN=z_segundo_y(testMNz)
#testMN
testN

In [ ]:
import copy
import numpy as np

def testfitN(testNu, n_classes, rulesNlistfitu):
    testN = copy.deepcopy(testNu)
    rulesNlistfit = copy.deepcopy(rulesNlistfitu)
    testfit = []
    

    for xi_key, xi_value in testN.items():
        testxi = []
        testxi.append(xi_key)
        x_ind = []

        for item in rulesNlistfit:
            x_ind.append(item[6])

        x_ind = [int(x) for x in x_ind]

        for i in range(n_classes):
            testxi.append([])

        for i in range(len(x_ind)):
            testxi[x_ind[i] + 1].append([])
        for second_key in xi_value:
            clatctri=[]
            sk = second_key
            for rule in rulesNlistfit:
                if rule[len("rulesN"):] == sk:
                    selected_rule = rule
            diclist = chamarvarvar(globals()[selected_rule])

            for dic in diclist:
                for key, value in dic.items():
                    for y_key, y_value in value.items():
                        cla=[]
                        ratio=[]
                        for y, coeficientes in y_value[0].items():
                            if y=='ratio':
                                ratio.append(coeficientes)
                            if y != 'tricluster_key' and y != 'ratio':
                                ai_tol = []
                                for coef, val in coeficientes['coeficientes'].items():
                                    ai_tol.append(val)
                                fit_coefficients_list = []  
                                y_values = [val[0] for val in testN[xi_key][sk][y]['values']]
                                x_values = [val[1] for val in testN[xi_key][sk][y]['values']]

                                
                                fit_coefficients = np.polyfit(x_values, y_values, len(ai_tol)-1)

                                
                                fit_coefficients_without_constant = fit_coefficients[:-1]

                                
                                fit_coefficients_list.append(fit_coefficients_without_constant)
                                
                                for ii, co in enumerate(fit_coefficients_list[0]):
                                    tol=ai_tol[-1]
                                    val=co
                                    valm=co+tol*co
                                    valmn=co-tol*co
                                    if ai_tol[ii]>=min(valm,valmn) and ai_tol[ii]<=max(valm,valmn):
                                        cla.append(1)
                                    else:
                                        cla.append(0)
                        
                        clatctri.append(ratio[0]) if sum(cla) == len(cla) else clatctri.append(0)
            testxi[int(sk[0])+1][int(sk[1:])-1].append(max(clatctri)) if sum(clatctri) > 0 else testxi[int(sk[0])+1][int(sk[1:])-1].append(0)
            
        testfit.append(testxi)
        

    organized_testNver = [[elemento[0], elemento[1:]] for elemento in testfit]                        
    return organized_testNver

testNver = testfitN(testN, n_classes, rulesNlistfit)
testNver

In [ ]:
import copy

def simplify_results(resultsu):
    results = copy.deepcopy(resultsu)
    for item in results:
        for sublist in item[1]:
            for i, subsublist in enumerate(sublist):
                if isinstance(subsublist, list) and len(subsublist) == 1:
                    sublist[i] = subsublist[0]
    return results

resultsf = simplify_results(testNver)

In [ ]:
import copy
import random
import numpy as np
from sklearn.metrics import confusion_matrix

def classificador(results_u, test_u):
    results = copy.deepcopy(results_u)
    test = copy.deepcopy(test_u)
    total_objetos = len(test)
    corretos = 0
    empates = 0
    empates_corretos = 0
    empates_incorretos = 0
    corretos_sem_empates = 0  
    incorretos_sem_empates = 0  

    
    true_labels = []
    predicted_labels = []

    for ii, val in enumerate(results):
        pontos_rec = []
        max_pontos = []  
        for ij, cla in enumerate(val[1]):
            pontos = sum(cla)
            max_pontos.append(len(cla))  
            pontos_rec.append(pontos)

        # Normalizando os pontos
        pontos_normalizados = [p/max_pontos[idx] for idx, p in enumerate(pontos_rec)]

       
        max_pontos_norm = max(pontos_normalizados)
        indices_max_pontos_norm = [i for i, j in enumerate(pontos_normalizados) if j == max_pontos_norm]

        # Verificar se há empate
        if len(indices_max_pontos_norm) > 1:
            empates += 1
            classe_predita = random.choice(indices_max_pontos_norm)
        else:
            classe_predita = indices_max_pontos_norm[0]

        
        classe_verdadeira = test.iloc[ii][' class']  
        if classe_predita == classe_verdadeira:
            corretos += 1
            if len(indices_max_pontos_norm) == 1:  
                corretos_sem_empates += 1
        else:
            if len(indices_max_pontos_norm) == 1:  
                incorretos_sem_empates += 1
        if len(indices_max_pontos_norm) > 1 and classe_verdadeira in indices_max_pontos_norm:
                empates_corretos += 1
        if len(indices_max_pontos_norm) > 1 and classe_verdadeira not in indices_max_pontos_norm:
            empates_incorretos += 1

       
        true_labels.append(classe_verdadeira)
        predicted_labels.append(classe_predita)

    
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    precisao = corretos / total_objetos
    precisao_empates_corretos = (corretos_sem_empates + empates_corretos) / total_objetos  
    precisao_empates_incorretos = (corretos_sem_empates) / total_objetos  

    return precisao, empates, precisao_empates_corretos, precisao_empates_incorretos, conf_matrix


resultados = classificador(resultsf, test)

In [ ]:
def add_true_class(resultd, test_df):
    result=copy.deepcopy(resultd)
    for item in result:
        object_name = item[0]
        true_class = test_df[test_df['X'] == object_name][' class'].values[0]
        item.append(true_class)
    return result


resultRF=add_true_class(resultsf, test)

In [ ]:
def organize_data(resultRF):
    
    data = {'Pacients': [], 'True Class': []}
    
    
    num_classes = len(resultRF[0][1])
    
    
    for i in range(num_classes):
        data[f'Class {i} Rules'] = []
    
    
    for item in resultRF:
        pacient = item[0]
        rules = item[1]
        true_class = item[2]
        
        data['Pacients'].append(pacient)
        data['True Class'].append(true_class)
        
        
        for i, rule_list in enumerate(rules):
            data[f'Class {i} Rules'].append(rule_list)
    
   
    df = pd.DataFrame(data)
    return df



rfdf = organize_data(resultRF)
rfdf

In [ ]:
import pandas as pd


def process_dataframe(rfdf,t):
    
    data = {'Pacients': rfdf['Pacients'], 'True Classe': rfdf['True Class']}
    
    
    sum_0 = []
    sum_1 = []

    for i in range(len(rfdf)):
        sum_0.append(sum(float(val) for val in rfdf.loc[i, 'Class 0 Rules'] if float(val) >= t[0]))


        sum_1.append(sum(float(val) for val in rfdf.loc[i, 'Class 1 Rules'] if float(val) >= t[1]))


    
    
    data['Sum 0'] = sum_0
    data['Sum 1'] = sum_1
    
    
    new_df = pd.DataFrame(data)
    
    
    new_df['Norm 0'] = new_df['Sum 0'] / new_df['Sum 0'].sum()
    new_df['Norm 1'] = new_df['Sum 1'] / new_df['Sum 1'].sum()
    
    
    new_df['Predicted'] = new_df[['Norm 0', 'Norm 1']].idxmax(axis=1).str[-1].astype(int)
    
    
    new_df['Ratio'] = (new_df['True Classe'] == new_df['Predicted']).astype(int)
        
    ratio_mean = new_df['Ratio'].sum() / len(new_df['Ratio'])
    
    return new_df,ratio_mean

exp=process_dataframe(rfdf,[0,0]) #Choose the values that had the highest Mean of Ration columns that is obtained below.
print(exp[1])
exp[0]

In [ ]:
t_values = [(i / 20, j / 20) for i in range(20) for j in range(20)]

previous_mean = 0

for t in t_values:
    exp, ratio_mean = process_dataframe(rfdf, t)
    if ratio_mean > previous_mean:
        print(t)
        print("Mean of Ratio column:", ratio_mean)
        print("\n")
        previous_mean = ratio_mean